In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os,sys, kimimaro
import numpy as np
from skimage.morphology import disk, ball, square
import scipy.ndimage as nd
import scipy.signal as sig
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import cloudvolume as cv
from IPython.display import HTML
from matplotlib.collections import LineCollection

sys.path.append("/home/phornauer/Git/axon_tracking/")
import axon_tracking.skeletonization as skel
import axon_tracking.template_extraction as te
import axon_tracking.visualization as vis

In [ ]:
params = dict()
params['noise_threshold'] = -1.5
params['abs_threshold'] = -0.2
params['upsample'] = [1,1,1]
params['sampling_rate'] = 20000/params['upsample'][2] # [Hz]
params['ms_cutout'] = [1.5, 5.0]
params['filter_footprint'] = ball(1)#np.ones((2,2,3))
params['max_velocity'] = 1 # [m/s]
params['max_noise_level'] = 0.25

In [ ]:
root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics/Large_dose_range/concatenated/'
#root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_3/concatenated/'
stream_id = 'well007'

In [ ]:
vis.plot_template_overview(root_path, stream_id, params, n_cols = 3, vmin=-10, vmax=0, filename='test',unit_ids= [],overwrite=False)

In [ ]:
np.mean(noise)

In [ ]:
np.min(capped_template)

In [ ]:
root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics/Large_dose_range/concatenated/'
#root_path = '/net/bs-filesvr02/export/group/hierlemann/intermediate_data/Maxtwo/phornauer/Chemogenetics_2/Week_3/concatenated/'
stream_id = 'well014'
template_id = 81
template, template_save_file, noise = skel.load_template_file(root_path, stream_id, template_id)
temp_diff = np.diff(template)
capped_template, target_coor= skel.localize_neurons(temp_diff, ms_cutout=params['ms_cutout'])

#tmp_filt = nd.median_filter(capped_template,footprint=params['filter_footprint'])
tmp_filt = nd.gaussian_filter(capped_template,sigma=1)

interp_temp = skel.interpolate_template(tmp_filt,spacing=params['upsample'])
th_template = skel.threshold_template(interp_temp, noise, target_coor, params)

vis.plot_template_and_noise(tmp_filt,noise,th_template)

In [ ]:
skel_params = dict()
skel_params['scale'] = 1#1
skel_params['const'] = 2#3
skel_params['dust_threshold'] = 5
skel_params['anisotropy'] = (10.0, 10.0, 10.0)
skel_params['tick_threshold'] = 5
skel_params['n_jobs'] = 16

In [ ]:
t_cap = [0, th_template.shape[2]] #in samples
skels = skel.skeletonize(th_template[:,:,t_cap[0]:t_cap[1]].astype('bool'), **skel_params)

skeleton = kimimaro.join_close_components(skels[1], radius=10)

skeleton = kimimaro.postprocess(skeleton, tick_threshold=5, dust_threshold=10)

skeleton = kimimaro.join_close_components(skeleton, radius=20)

#skeleton = kimimaro.postprocess(skeleton, tick_threshold=5, dust_threshold=100)

skeleton = skeleton.consolidate()
skel_paths = cv.Skeleton.paths(skeleton)

In [ ]:
vis.plot_delay_skeleton(skel_paths, params, skel_params,figsize=4, plot_ais=False, plot_ais_connection=False)

In [ ]:
x,y,z = vis.plot_skeleton(skeleton, marker_size=3,fig_size=4,ais=[])

In [ ]:
qc_params = dict()
qc_params['window_size'] = 5
qc_params['max_duplicate_ratio'] = 0.3
qc_params['min_r2'] = 0.8
qc_params['vel_range'] = [0.2, 0.8]
qc_params['min_length'] = 1

In [ ]:
all_branches = skel.branches_from_paths(skeleton)
scaled_qc_list, r2s, vels, lengths = skel.perform_path_qc(all_branches, params,**qc_params)

In [ ]:
vis.plot_conduction_velocity(scaled_qc_list,params)

In [ ]:
vis.plot_velocity_qc(vels,r2s,fig_size=(6,2))

In [ ]:
vis.plot_delay_skeleton(scaled_qc_list, params, skel_params,figsize=4, plot_ais=False, plot_ais_connection=False)

In [ ]:
test_skels = [skeleton.from_path(x) for x in scaled_qc_list]

In [ ]:
test_skeleton = skeleton.simple_merge(test_skels)
test_skeleton = kimimaro.join_close_components(test_skeleton,radius=100)
test_skeleton = kimimaro.postprocess(test_skeleton, dust_threshold=500, tick_threshold=10)
test_skeleton = test_skeleton.consolidate()

In [ ]:
vis.plot_delay_skeleton(test_skeleton.paths(), params, skel_params,figsize=4, plot_ais=False, plot_ais_connection=False)

In [ ]:
all_branches = skel.branches_from_paths(test_skeleton)
scaled_qc_list, r2s, vels, lengths = skel.perform_path_qc(all_branches, params,**qc_params)

In [ ]:
vis.plot_velocity_qc(vels,r2s,fig_size=(6,2))

In [ ]:
[vels[x] for x in range(len(vels)) if r2s[x] >0.8]

In [ ]:
ani = vis.generate_propagation_gif(interp_temp, cumulative=False, vertices=[], downsample=2, clim=[-2, -0.15], cmap="Greys", marker_size=0.1)

In [ ]:
HTML(ani.to_jshtml())

In [ ]:
save_path = os.path.join(root_path,stream_id,str(template_id)+'_delay.png')
vis.plot_delay_contour(capped_template,test_skeleton,params,skel_params,radius=5,save_path=save_path)

In [ ]:
save_path = os.path.join(root_path,stream_id,str(template_id)+'.png')
save_path

In [ ]:
vis.plot_filled_contour(capped_template,test_skeleton,params,radius=5,save_path=save_path)

In [ ]:
interp_tmp = skel.interpolate_template(capped_template,spacing=params['upsample'])

In [ ]:
sorted_vertices = skel.path_to_vertices(skel.scale_path_coordinates(test_skeleton.paths()))

In [ ]:
sorted_vertices = skel.path_to_vertices(test_skeleton.paths())

In [ ]:
skel_mat = np.zeros(interp_tmp.shape)
skel_mat[tuple(sorted_vertices.astype('int').T)] = True

In [ ]:
dil_mat = nd.binary_dilation(skel_mat,structure=ball(10))
th_data = (interp_tmp * dil_mat)#[:,:,t_cap[0]:t_cap[1]]
contour_data = np.abs(np.min(th_data,axis=2).T)
#contourf_lines = np.linspace(-np.max(contour_data),-0.1,500)

In [ ]:
contourf_lines = np.geomspace(-np.max(contour_data), -0.2,20)
np.concatenate(contourf_lines, -1)

In [ ]:
fig, ax = plt.subplots(figsize=(22,12))
plt.contourf(-contour_data,levels = contourf_lines,cmap="inferno",vmin=-10,vmax=-0.1)# ,linewidths = 0.2,vmax=20,vmin=2)hatches =[':'],
ax.autoscale_view()
ax.set_ylim(ax.get_ylim()[::-1])
plt.colorbar()
plt.show()

In [ ]:
pre = test_skeleton.to_precomputed()

In [ ]:
cv.Skeleton.from_precomputed(pre)

In [ ]:
import pickle
with open('skel.pkl', 'wb') as file: 
      
    # A new file will be created 
    pickle.dump(test_skeleton, file) 

In [ ]:
with open('skel.pkl', 'rb') as file: 
      
    # Call load method to deserialze 
    myvar = pickle.load(file) 

In [ ]:
test_skeleton.vertices

In [ ]:
np.min(skeleton.vertices,axis=0)

In [ ]:
contour_lines = np.geomspace(0.2,np.max(contour_data),20)

fig, ax = plt.subplots()
plt.contour(contour_data,levels = contour_lines,colors='k',vmin=-5,vmax=-0.1,linewidths = 0.1,alpha=0.5)#,vmax=20,vmin=2)hatches =[':'],
ax.autoscale_view()
ax.set_ylim(ax.get_ylim()[::-1])
plt.show()

In [ ]:
contour_lines = np.geomspace(0.2,np.max(contour_data),20)
#unscaled_qc_2_list = skel.scale_path_coordinates(unscaled_qc_2_list)

fig, ax = vis.plot_delay_skeleton(skel.unscale_path_coordinates(test_skeleton.paths()), params, skel_params,figsize=6, plot_ais=True, plot_ais_connection=False, linewidth=2)
plt.contour(contour_data,levels = contour_lines,colors='k',vmin=-5,vmax=-0.2,linewidths = 0.1,alpha=0.7,zorder=0)#,vmax=20,vmin=2)hatches =[':'],
fig.set_figwidth(12)
ax.autoscale_view()
#ax.set_ylim(ax.get_ylim()[::-1])
ax.set_ylim([0,120])
plt.show()